# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [4]:
from collections import deque
import heapq
import itertools

class State:
    def __init__(self, positions, boat):
        # positions: dict mapping 'H1','W1',... to 'L','I','R'
        self.positions = positions
        self.boat = boat

    def __hash__(self):
        items = tuple(sorted(self.positions.items()))
        return hash((items, self.boat))

    def __eq__(self, other):
        return isinstance(other, State) and self.boat == other.boat and self.positions == other.positions

    def is_valid(self):
        # For each location, enforce jealousy constraint
        for loc in ('L', 'I', 'R'):
            wives = {p for p, pos in self.positions.items() if p.startswith('W') and pos == loc}
            husbands = {p for p, pos in self.positions.items() if p.startswith('H') and pos == loc}
            for w in wives:
                idx = w[1:]
                if f'H{idx}' not in husbands and husbands:
                    return False
        return True

    def is_goal(self):
        # everyone on right bank
        return all(pos == 'R' for pos in self.positions.values())

    def successors(self):
        # possible boat moves: L <-> I, I <-> R
        neighbors = {'L': ['I'], 'I': ['L', 'R'], 'R': ['I']}
        people_here = [p for p, pos in self.positions.items() if pos == self.boat]
        for dest in neighbors[self.boat]:
            for k in (1, 2):
                for combo in itertools.combinations(people_here, k):
                    new_pos = self.positions.copy()
                    for p in combo:
                        new_pos[p] = dest
                    new_state = State(new_pos, dest)
                    if new_state.is_valid():
                        yield new_state, combo, dest

    def __str__(self):
        pos_str = ', '.join(f"{p}:{pos}" for p, pos in sorted(self.positions.items()))
        return f"Boat at {self.boat}; {pos_str}"

# Depth-First Search (iterative)
def dfs(initial):
    stack = [(initial, [])]
    visited = set()
    while stack:
        state, path = stack.pop()
        if state.is_goal():
            return path + [(state, None)]
        if state in visited:
            continue
        visited.add(state)
        for succ, moved, dest in state.successors():
            stack.append((succ, path + [(state, (moved, dest))]))
    return None

# Backtracking (recursive DFS)
def backtrack(state, path, visited):
    if state.is_goal():
        return path + [(state, None)]
    visited.add(state)
    for succ, moved, dest in state.successors():
        if succ not in visited:
            res = backtrack(succ, path + [(state, (moved, dest))], visited)
            if res:
                return res
    visited.remove(state)
    return None

# A* search with tie-breaker counter
def heuristic(state):
    remaining = sum(1 for pos in state.positions.values() if pos != 'R')
    return (remaining + 1) // 2


def astar(initial):
    counter = itertools.count()
    open_set = []  # entries: (f, count, g, state, path)
    g_scores = {initial: 0}
    heapq.heappush(open_set, (heuristic(initial), next(counter), 0, initial, []))
    visited = set()
    while open_set:
        f, _, g, state, path = heapq.heappop(open_set)
        if state.is_goal():
            return path + [(state, None)]
        if state in visited:
            continue
        visited.add(state)
        for succ, moved, dest in state.successors():
            tg = g + 1
            if succ in visited and tg >= g_scores.get(succ, float('inf')):
                continue
            if tg < g_scores.get(succ, float('inf')):
                g_scores[succ] = tg
                fs = tg + heuristic(succ)
                heapq.heappush(open_set, (fs, next(counter), tg, succ, path + [(state, (moved, dest))]))
    return None

# Helper to construct initial state
def make_initial(n):
    pos = {f'H{i}': 'L' for i in range(1, n+1)}
    pos.update({f'W{i}': 'L' for i in range(1, n+1)})
    return State(pos, 'L')

if __name__ == '__main__':
    n = 3
    initial = make_initial(n)

    # DFS
    print("Solving with DFS...")
    path = dfs(initial)
    for step, (state, move) in enumerate(path, 1):
        if move:
            people, dest = move
            origin = state.boat
            print(f"Step {step}: Move {people} from {origin} to {dest}")

    # Backtracking
    print("\nSolving with Backtracking...")
    path = backtrack(initial, [], set())
    for step, (state, move) in enumerate(path, 1):
        if move:
            people, dest = move
            origin = state.boat
            print(f"Step {step}: Move {people} from {origin} to {dest}")

    # A*
    print("\nSolving with A*...")
    path = astar(initial)
    for step, (state, move) in enumerate(path, 1):
        if move:
            people, dest = move
            origin = state.boat
            print(f"Step {step}: Move {people} from {origin} to {dest}")

    print("\nAll solutions computed.")

Solving with DFS...
Step 1: Move ('W2', 'W3') from L to I
Step 2: Move ('W2', 'W3') from I to R
Step 3: Move ('W3',) from R to I
Step 4: Move ('W3',) from I to L
Step 5: Move ('W1', 'W3') from L to I
Step 6: Move ('W1', 'W3') from I to R
Step 7: Move ('W2', 'W3') from R to I
Step 8: Move ('W3',) from I to R
Step 9: Move ('W1', 'W3') from R to I
Step 10: Move ('W2', 'W3') from I to R
Step 11: Move ('W2',) from R to I
Step 12: Move ('W1', 'W2') from I to L
Step 13: Move ('H2', 'W2') from L to I
Step 14: Move ('H2',) from I to L
Step 15: Move ('H2', 'H3') from L to I
Step 16: Move ('H2', 'H3') from I to R
Step 17: Move ('W3',) from R to I
Step 18: Move ('W2', 'W3') from I to R
Step 19: Move ('H3', 'W3') from R to I
Step 20: Move ('H3',) from I to R
Step 21: Move ('H2', 'H3') from R to I
Step 22: Move ('W3',) from I to R
Step 23: Move ('W2', 'W3') from R to I
Step 24: Move ('H3', 'W3') from I to L
Step 25: Move ('H1', 'W1') from L to I
Step 26: Move ('W1', 'W2') from I to R
Step 27: Move (